In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from env import user, password, host
from scipy.stats import levene, ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import math
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('CAMPUS_summary_22.csv')
df21 = pd.read_csv('CAMPUS_summary_21.csv')
df20 = pd.read_csv('CAMPUS_summary_20.csv')
df19 = pd.read_csv('CAMPUS_summary_19.csv')
df18 = pd.read_csv('CAMPUS_summary_18.csv')

In [3]:
df=df.rename(columns={'AGGREGATION LEVEL': 'agg_level', 'CAMPUS':'campus_number', 'REGION': 'region', 'DISTRICT NAME AND NUMBER': 'dist_name_num','CHARTER_STATUS':'charter_status', 'CAMPUS NAME AND NUMBER': 'campus_name_num', 'SECTION': 'section', 'HEADING':'heading','HEADING NAME': 'heading_name', 'YR22':'student_count'})

In [5]:
df['charter_encoded'] = df.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 'TRADITIONAL ISD/CSD':0})

In [7]:
df=df[(df.heading == 'A01') | (df.heading ==  'A03')]
    

In [9]:
df=df[df['student_count'] != '-999']

In [11]:
df['student_count']= df['student_count'].str.replace("<", "")

In [13]:
df['student_count'] = df['student_count'].astype(float)

In [15]:
dfpivot=df.pivot(index='campus_number', columns='heading', values= 'student_count').dropna()

In [16]:
dfpivot

heading,A01,A03
campus_number,,
1902001,173.0,37.0
1903001,335.0,141.0
1903041,309.0,65.0
1903102,295.0,40.0
1904001,248.0,101.0
...,...,...
253901001,1028.0,239.0
253901041,856.0,298.0
253901106,516.0,13.0


In [17]:
df=df.merge(dfpivot,how= 'right', on= 'campus_number')

In [19]:
df=df.rename(columns={'A01': 'student_enrollment', 'A03':'discipline_count'})

In [21]:
df['discipline_percent']= ((df['discipline_count']/df['student_enrollment'])*100)

In [23]:
df=df.round({'discipline_percent': 0})

In [26]:
df=df.drop(columns=['agg_level', 'campus_number', 'region', 'charter_status', 'dist_name_num', 'section', 'heading',
       'heading_name', 'student_count'])

In [ ]:
def split_tea_data(df):
    '''
    This function performs split on tea data, stratify charter_encoded.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.charter_encoded)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.charter_encoded)
    return train, validate, test

In [ ]:
train, validate, test= split_tea_data() 

In [ ]:
train.head()

In [ ]:
def prep(df):
    global df22
    df22=df22.rename(columns={'AGGREGATION LEVEL': 'agg_level', 'CAMPUS':'campus_number', 'REGION': 'region', 'DISTRICT NAME AND NUMBER': 'dist_name_num','CHARTER_STATUS':'charter_status', 'CAMPUS NAME AND NUMBER': 'campus_name_num', 'SECTION': 'section', 'HEADING':'heading','HEADING NAME': 'heading_name', 'YR22':'student_count'})
    df22['charter_encoded'] = df22.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 'TRADITIONAL ISD/CSD':0})
    df22=df22[(df22.heading == 'A01') | (df22.heading ==  'A03')]
    df22 = df22[df22['student_count'] != '-999']
    df22.dropna()
    df22=df22.drop_duplicates()
    df22pivot=df22.pivot(index='campus_number', columns='heading_name', values= 'student_count').dropna()
    df22=df22.merge(df22pivot,how= 'right', on= 'campus_number')
    df22=df22.drop(columns=[ 'agg_level', 'region','charter_status'])
    # Removed values that were masked FERPA data
    df22['student_count']= df22['student_count'].str.replace("<", "")
    # Converted column to float
    df22['student_count'] = df22['student_count'].astype(float)
    df22=df22.drop_duplicates()
    df22.dropna()
    df22=df22.rename(columns={'CAMPUS CUMULATIVE YEAR END ENROLLMENT': 'student_enrollment', 
                              'CAMPUS DISCIPLINE RECORD COUNT':'discipline_count'})
    df22['student_enrollment']= df22['student_enrollment'].str.replace("<", "")
    df22['discipline_count']= df22['discipline_count'].str.replace("<", "")
    df22['student_enrollment'] = df22['student_enrollment'].astype(float)
    df22['discipline_count'] = df22['discipline_count'].astype(float)
    df22['discipline_percent']= ((df22['discipline_count']/df22['student_enrollment'])*100)
    df22=df22.round({'discipline_percent': 0})
    df22=df22.reset_index(drop=True)